- $G_t = \sum_{k=t+1}^T γ^{k-t} r_k$
- $r_t(θ) = \frac{π_θ(a_t|s_t)}{π_{θ_\text{old}}(a_t|s_t)}$
- PG vs. REINFORCE
$$\nabla_\theta J(\theta)=\mathbb E_{\tau\sim\pi_\theta}[G_t\nabla_\theta \log\pi_\theta(A_t|S_t)]$$
    - PG: $G(\tau)=r_0+\gamma r_1+\gamma^2 r_2+\cdots=\sum_{t=0}^T\gamma^tr_t$
        - $G(\tau)\nabla_\theta\log\pi_\theta(A_t|S_t)$，无论在哪个时刻 $t$，我们始终都会使用固定不变的权重 $G(\tau)$ 来增加或减少采取行动 $A_t$ 的条件概率；
        - 存在严重的奖励分配的问题
    - REINFORCE: $G_t=\sum_{k=t+1}^{T}\gamma^{k-t}r_k$
        - $∇θ J(θ) = E_{τ∼π_θ}[ ∇_θ \log π_θ(a_t|s_t) G_t ]$
            - 这里的 $G_t$ 是从蒙特卡洛采样中直接得到的，方差很高。
        - agent 行动的好坏是根据行动之后获得的奖励综合来评估的，采取某个行动之前获得的奖励与该行动的好坏无关；
- A2C (advantage actic-critic)
    - minus baseline
      - 在一些坏的局势下，做什么动作都会得到负的奖励。所以要减去baseline。（即坏的局势下，仍然有好的 action，比如走出当前困境的 action）
- PPO
    - $L^{PPO}(θ, φ) = E_t [ L_t^{CLIP}(θ) - c_1 L_t^{VF}(φ) + c_2 S(π_θ) ]$
        - $L_t^{CLIP}(θ) = \min( r_t(θ) A_t, \text{clip}(r_t(θ), 1-ϵ, 1+ϵ) A_t )$
            - $A_t^{GAE} = ∑_{l=0}^{T-t-1} (γλ)^l δ_{t+l}$ ，其中 $δ_t = r_t + γV_φ(s_{t+1}) - V_φ(s_t)$
        - 价值函数损失 (Value Function Loss)： $L_t^{VF}(φ) = (V_φ(s_t) - G_t^{target})^2$ 用于更新 Critic 网络自身。
        - 熵奖励 (Entropy Bonus) S：鼓励探索。

## PG

$$
\mathbb E_{\tau \sim p_\theta(\tau)}[R(\tau)]
$$

- 关于 $R(\tau) = \sum_{t=1}^{T} r_t$:
    - return to go: $\hat{R}_t = \sum_{t'=t}^{T} r_{t'}$
    - Discount Factor: $\hat{R}_t = \sum_{t'=t}^{T} \gamma^{t'-t} r_{t'}$
    - Baseline: $\text{回报项} = \hat{R}_t - b(s_t)$
- 求其 gradient
$$
\begin{split}
\sum_{\tau}R(\tau)\nabla p_\theta(\tau)&=\sum_\tau R(\tau) p_\theta(\tau)\nabla \log p_\theta(\tau)\\
&=\mathbb E_{\tau\sim p_\theta(\tau)}R(\tau)\nabla \log p_\theta(\tau)
\end{split}
$$
- 对 $\nabla \log p_\theta(\tau)$ 进行展开
$$
\begin{split}
\nabla \log p_\theta(\tau)&=\nabla \log\left(p(s_1)\prod p(a_t|s_t)\prod p(s_{t+1}|s_t,a_t)\right)\\
&=\nabla \left(\log p(s_1)+\sum \log p(a_t|s_t)+\sum \log p(s_{t+1}|s_t,a_t)\right)\\
&=\sum_{t=1}^T\nabla \log p(a_t|s_t)
\end{split}
$$
- 因此
$$
\begin{split}
\sum_{\tau}R(\tau)\nabla p_\theta(\tau)&=\sum_\tau R(\tau) p_\theta(\tau)\nabla \log p_\theta(\tau)\\
&=\mathbb E_{\tau\sim p_\theta(\tau)}R(\tau)\nabla \log p_\theta(\tau)\\
&\approx\frac1N\sum_{n=1}^NR(\tau^n)\nabla \log p_\theta(\tau^n)\\
&=\frac1N\sum_{n=1}^N\sum_{t=1}^{T_n}R(\tau^n)\nabla \log p_\theta(a_t^n|s_t^n)
\end{split}
$$


## REINFORCE++

- https://hijkzzz.notion.site/reinforce-plus-plus
- 在不引入 Critic 网络的情况下，借鉴 PPO 的稳定性技术，从而实现比 PPO 更高效、比 REINFORCE 更稳定的训练。
- 核心思想：做减法和加法。
    - 减法（丢弃的）：它丢弃了 Critic 网络。这是它计算效率提升的根本原因。
    - 加法（借鉴的）：它从 PPO 和其他 RLHF 实践中“借”来了多个关键的优化技巧：

$$
L^{REINFORCE++}(θ) = E_t [\min(r_t(θ) A_t^{norm}, clip(r_t(θ), 1-ϵ, 1+ϵ) A_t^{norm} ) ]
$$

这里的关键区别在于 $A_t^{norm}$ 的计算：

- 定义单步奖励：首先，它在奖励中直接融入 KL 惩罚。根据论文，这个奖励 $r(x,y)$ 只在序列末端（EOS token）给出。
    - $r'(s_t, a_t) = I(s_t=\text{EOS})r(x,y) - β KL(π_θ(·|s_t) || π_{SFT}(·|s_t))$
    - `[小惩罚, 小惩罚, ..., 小惩罚, 最终大奖+小惩罚]。`
- 计算“伪优势”：它计算一个从当前步到结尾的累积奖励作为“伪优势” $A_t$。
    - $A_t(s_t, a_t) = r(x,y) - β ∑_{i=t}^T KL(π_θ(·|s_i) || π_{SFT}(·|s_i))$
        - $A_t = r'(s_t, a_t) + r'(s_{t+1}, a_{t+1}) + ... + r'(s_T, a_T)$ ($γ=1$)
    - 在标准的强化学习中，“优势函数 (Advantage)”的定义是 $A(s,a) = Q(s,a) - V(s)$，即某个动作的价值减去当前状态的平均价值。它衡量的是一个动作比“平均水平”好多少。 而 REINFORCE++ 里的 $A_t$ 并没有减去一个基线（Baseline）或价值函数 $V(s)$，它就是对未来奖励的原始累加。所以我们称之为“伪优势”，它只是一个中间计算值。
- 进行归一化：这是替代 Critic 的关键步骤。对一个批次（batch）内计算出的所有 $A_t$ 进行 Z-score 归一化。
    - $A_t^{norm} = \frac{A_t - μ_A}{σ_A + ε_{tiny}}$ 其中 μ_A 和 σ_A 分别是当前批次中所有 A_t 的均值和标准差。

| 特性 | REINFORCE | PPO (Proximal Policy Optimization) | REINFORCE++ |
| :--- | :--- | :--- | :--- |
| **基本思想** | 基础策略梯度 | Actor-Critic，带信任域约束 | 借鉴了 PPO 技巧的无 Critic 策略梯度 |
| **方差缩减** | 无（或仅有简单基线） | **Critic 网络**计算优势函数 `A(s,a)` | **优势归一化** (Z-score Normalization) |
| **策略更新约束** | 无 | **PPO-Clip 损失函数** | **PPO-Clip 损失函数** |
| **核心组件** | 策略网络 (Actor) | 策略网络 (Actor) + **价值网络 (Critic)** | 策略网络 (Actor) |
| **计算复杂度** | 低 | 高 | 中等（显著低于 PPO） |
| **稳定性** | 低 | 高 | 中等（显著高于 REINFORCE） |

### norm

让我们参考论文中的超参数来构建一个真实的场景：
- Rollout Batch Size: 256
- Samples per Prompt: 4

这意味着在一个“Rollout”阶段，我们有 256 / 4 = 64 个不同的 prompt。

- 对于 Prompt 1，生成了：回答1A, 回答1B, 回答1C, 回答1D。
- 对于 Prompt 2，生成了：回答2A, 回答2B, 回答2C, 回答2D。
- ...
- 对于 Prompt 64，生成了：回答64A, 回答64B, 回答64C, 回答64D。

256 * 200 = 51200 个 A_t, 
- “拉平”数据: `All_A_values = [A_{1A, 1}, A_{1A, 2}, ..., A_{1A, 150}, A_{1B, 1}, ..., A_{64D, 210}]`
- 计算统计量: 对这个包含 51,200 个数值的 All_A_values 列表，我们计算它的总均值 $μ_A$ 和总标准差 $σ_A$。
    - 这个 $μ_A$ 非常有意义，它代表了在当前这个批次的所有情境下（不同 prompt，不同回答），一个 token 平均能带来的未来期望收益。它是一个动态的、基于当前批次数据的全局基线。
 
计算最终的 A_t^{norm}
- 对“回答1A”的 token 1: $A_{1A,1}^{norm} = \frac{A_{1A,1} - μ_A}{σ_A}$
- 对“回答64D”的 token 210: $A_{64D,210}^{norm} = \frac{A_{210,64D} - μ_A}{σ_A}$

## RLHF


$$
r(s_t,a_t)=\mathbb I(s_t=[\text{EOS}])r(x,y)-\beta KL(t)
$$

- RLHF 中的 reward 定义在 response 级别，而非中间的 token 级别
    - 中间的 token reward 默认为 0
    - reward + kl-penalty
- Advantage
    - $\delta_t^V=r_t+\gamma V_\theta(s_{t+1})-V_\theta(s_t)$
    - $A_\theta^{GAE}(s_t,a)=\delta_t^V+\gamma\lambda A_\theta^{GAE}(s_{t+1},a)$
- returns = advantages + values
    - δt = rt + γ·V(st+1) - V(st)
    - At = δt + (γλ)At+1


```python
def compute_gae_advantage_return(token_level_rewards: torch.Tensor, values: torch.Tensor, eos_mask: torch.Tensor, gamma: torch.Tensor, lam: torch.Tensor):
    with torch.no_grad():
        lastgaelam = 0
        advantages_reversed = []
        gen_len = token_level_rewards.shape[-1]

        for t in reversed(range(gen_len)):
            nextvalues = values[:, t + 1] if t < gen_len - 1 else 0.0
            delta = token_level_rewards[:, t] + gamma * nextvalues - values[:, t]
            lastgaelam = delta + gamma * lam * lastgaelam
            advantages_reversed.append(lastgaelam)
        advantages = torch.stack(advantages_reversed[::-1], dim=1)

        returns = advantages + values
        advantages = verl_F.masked_whiten(advantages, eos_mask)
    return advantages, returns
```